In [1]:
import GetOldTweets3 as got3
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import time

In [2]:
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 350)

I will extract top tweets for the past 10 years. 100 top tweets will be extracted each month on the specified topic for past 10 years

In [3]:

def get_tweets(topic, end_date, years=13):
    columns = ['id', 'url', 'author', 'retweets', 'favorites', 'mentions', 'hashtags', 'geo', 'time', 'text']
    text_query = topic
    
    # convert the end_date string to the datetime object
    end_date_datetime = datetime.strptime(end_date, '%Y-%m-%d')

    # This section is to create a list of datetime objects that are 1 month apart going backwards
    date_range = [end_date_datetime]
    total_days = 365 * years
    
    prior_date = end_date_datetime
    
    while total_days > 0:
        prior_date = prior_date - relativedelta(days=1)
        date_range.append(prior_date)
        total_days -= 1
        
    # Convert the elements in the list from datetime objects to string
    date_range_string = [twitter_date.strftime('%Y-%m-%d') for twitter_date in date_range][::-1]
    print("Start querying data...")
    
    # Initialize an empty list to store dataframe from each iteration
    compiled_tweets_df = pd.DataFrame(columns=columns)
    
    
    for i in range(len(date_range_string)-1):
        tweetCriteria = got3.manager\
                .TweetCriteria()\
                .setQuerySearch(text_query)\
                .setLang('en')\
                .setSince(date_range_string[i])\
                .setUntil(date_range_string[i+1])\
                .setTopTweets(True)\
                .setMaxTweets(4)\
                .setEmoji("unicode")

        tweets = got3.manager.TweetManager.getTweets(tweetCriteria)

        text_tweets = [[tweet.id, tweet.permalink, tweet.username, tweet.retweets, 
                tweet.favorites, tweet.mentions, tweet.hashtags, tweet.geo, 
                tweet.date, tweet.text] for tweet in tweets]

        sample_tweets = pd.DataFrame(text_tweets, columns=columns)
        compiled_tweets_df = pd.concat([compiled_tweets_df, sample_tweets])
        print("Query between {} and {} complete!".format(date_range_string[i], date_range_string[i+1]))
        print("Pause the operation...")
        time.sleep(3) # pause for 2 seconds to avoid server crashing
        print("Begin new iteration...")
        
    return compiled_tweets_df # concatenate the list of dataframe into a single dataframe
    print("Operation complete!")
    

In [4]:
tweets = get_tweets("immigration", "2019-12-31")

Start querying data...
Query between 2007-01-03 and 2007-01-04 complete!
Pause the operation...
Begin new iteration...
Query between 2007-01-04 and 2007-01-05 complete!
Pause the operation...
Begin new iteration...
Query between 2007-01-05 and 2007-01-06 complete!
Pause the operation...
Begin new iteration...
Query between 2007-01-06 and 2007-01-07 complete!
Pause the operation...
Begin new iteration...
Query between 2007-01-07 and 2007-01-08 complete!
Pause the operation...
Begin new iteration...
Query between 2007-01-08 and 2007-01-09 complete!
Pause the operation...
Begin new iteration...
Query between 2007-01-09 and 2007-01-10 complete!
Pause the operation...
Begin new iteration...
Query between 2007-01-10 and 2007-01-11 complete!
Pause the operation...
Begin new iteration...
Query between 2007-01-11 and 2007-01-12 complete!
Pause the operation...
Begin new iteration...
Query between 2007-01-12 and 2007-01-13 complete!
Pause the operation...
Begin new iteration...
Query between 200

Begin new iteration...
Query between 2007-03-30 and 2007-03-31 complete!
Pause the operation...
Begin new iteration...
Query between 2007-03-31 and 2007-04-01 complete!
Pause the operation...
Begin new iteration...
Query between 2007-04-01 and 2007-04-02 complete!
Pause the operation...
Begin new iteration...
Query between 2007-04-02 and 2007-04-03 complete!
Pause the operation...
Begin new iteration...
Query between 2007-04-03 and 2007-04-04 complete!
Pause the operation...
Begin new iteration...
Query between 2007-04-04 and 2007-04-05 complete!
Pause the operation...
Begin new iteration...
Query between 2007-04-05 and 2007-04-06 complete!
Pause the operation...
Begin new iteration...
Query between 2007-04-06 and 2007-04-07 complete!
Pause the operation...
Begin new iteration...
Query between 2007-04-07 and 2007-04-08 complete!
Pause the operation...
Begin new iteration...
Query between 2007-04-08 and 2007-04-09 complete!
Pause the operation...
Begin new iteration...
Query between 200

Begin new iteration...
Query between 2007-06-24 and 2007-06-25 complete!
Pause the operation...
Begin new iteration...
Query between 2007-06-25 and 2007-06-26 complete!
Pause the operation...
Begin new iteration...
Query between 2007-06-26 and 2007-06-27 complete!
Pause the operation...
Begin new iteration...
Query between 2007-06-27 and 2007-06-28 complete!
Pause the operation...
Begin new iteration...
Query between 2007-06-28 and 2007-06-29 complete!
Pause the operation...
Begin new iteration...
Query between 2007-06-29 and 2007-06-30 complete!
Pause the operation...
Begin new iteration...
Query between 2007-06-30 and 2007-07-01 complete!
Pause the operation...
Begin new iteration...
Query between 2007-07-01 and 2007-07-02 complete!
Pause the operation...
Begin new iteration...
Query between 2007-07-02 and 2007-07-03 complete!
Pause the operation...
Begin new iteration...
Query between 2007-07-03 and 2007-07-04 complete!
Pause the operation...
Begin new iteration...
Query between 200

Begin new iteration...
Query between 2007-09-18 and 2007-09-19 complete!
Pause the operation...
Begin new iteration...
Query between 2007-09-19 and 2007-09-20 complete!
Pause the operation...
Begin new iteration...
Query between 2007-09-20 and 2007-09-21 complete!
Pause the operation...
Begin new iteration...
Query between 2007-09-21 and 2007-09-22 complete!
Pause the operation...
Begin new iteration...
Query between 2007-09-22 and 2007-09-23 complete!
Pause the operation...
Begin new iteration...
Query between 2007-09-23 and 2007-09-24 complete!
Pause the operation...
Begin new iteration...
Query between 2007-09-24 and 2007-09-25 complete!
Pause the operation...
Begin new iteration...
Query between 2007-09-25 and 2007-09-26 complete!
Pause the operation...
Begin new iteration...
Query between 2007-09-26 and 2007-09-27 complete!
Pause the operation...
Begin new iteration...
Query between 2007-09-27 and 2007-09-28 complete!
Pause the operation...
Begin new iteration...
Query between 200

Begin new iteration...
Query between 2007-12-13 and 2007-12-14 complete!
Pause the operation...
Begin new iteration...
Query between 2007-12-14 and 2007-12-15 complete!
Pause the operation...
Begin new iteration...
Query between 2007-12-15 and 2007-12-16 complete!
Pause the operation...
Begin new iteration...
Query between 2007-12-16 and 2007-12-17 complete!
Pause the operation...
Begin new iteration...
Query between 2007-12-17 and 2007-12-18 complete!
Pause the operation...
Begin new iteration...
Query between 2007-12-18 and 2007-12-19 complete!
Pause the operation...
Begin new iteration...
Query between 2007-12-19 and 2007-12-20 complete!
Pause the operation...
Begin new iteration...
Query between 2007-12-20 and 2007-12-21 complete!
Pause the operation...
Begin new iteration...
Query between 2007-12-21 and 2007-12-22 complete!
Pause the operation...
Begin new iteration...
Query between 2007-12-22 and 2007-12-23 complete!
Pause the operation...
Begin new iteration...
Query between 200

...

Begin new iteration...
Query between 2013-11-01 and 2013-11-02 complete!
Pause the operation...
Begin new iteration...
Query between 2013-11-02 and 2013-11-03 complete!
Pause the operation...
Begin new iteration...
Query between 2013-11-03 and 2013-11-04 complete!
Pause the operation...
Begin new iteration...
Query between 2013-11-04 and 2013-11-05 complete!
Pause the operation...
Begin new iteration...
Query between 2013-11-05 and 2013-11-06 complete!
Pause the operation...
Begin new iteration...
Query between 2013-11-06 and 2013-11-07 complete!
Pause the operation...
Begin new iteration...
Query between 2013-11-07 and 2013-11-08 complete!
Pause the operation...
Begin new iteration...
Query between 2013-11-08 and 2013-11-09 complete!
Pause the operation...
Begin new iteration...
Query between 2013-11-09 and 2013-11-10 complete!
Pause the operation...
Begin new iteration...


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



An error occured during an HTTP request: Remote end closed connection without response
Try to open in browser: https://twitter.com/search?q=immigration%20since%3A2013-11-10%20until%3A2013-11-11&src=typd
Traceback (most recent call last):
  File "D:\Anaconda_Navigator\envs\computational_content_analysis\lib\site-packages\GetOldTweets3\manager\TweetManager.py", line 343, in getJsonResponse
    response = opener.open(url)
  File "D:\Anaconda_Navigator\envs\computational_content_analysis\lib\urllib\request.py", line 526, in open
    response = self._open(req, data)
  File "D:\Anaconda_Navigator\envs\computational_content_analysis\lib\urllib\request.py", line 544, in _open
    '_open', req)
  File "D:\Anaconda_Navigator\envs\computational_content_analysis\lib\urllib\request.py", line 504, in _call_chain
    result = func(*args)
  File "D:\Anaconda_Navigator\envs\computational_content_analysis\lib\urllib\request.py", line 1361, in https_open
    context=self._context, check_hostname=self._ch

SystemExit: 

C:\Users\John Xu\AppData\Roaming\Python\Python36\site-packages\IPython\core\interactiveshell.py:3327: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
tweets.to_csv("immigration_tweets_data_daily.csv")

In [ ]:
tweets["text"].iloc